# Automated model training and evaluation using
* #### Scikit-learn
* #### Google Cloud Composer
* #### Goolge Cloud AI Platform Training
* #### MLflow

This notebook goes through these major steps:
* Step 1: Creates Cloud AI Platrform Training package for Chicago taxi fare predicion model.
* Step 2: Creates and deploy Airflow DAG to manage training process

> Costs. You migth be charged for the operations in this tutorial. Refer to the [Cloud AI Platform Training pricing page](https://cloud.google.com/ai-platform/training/pricing) for more information.

In [ ]:
import os
from IPython.core.display import display, HTML
import mlflow
import pymysql
from datetime import datetime

In [ ]:
# Jupyter magic jinja template to create Python file with variable substitution.
# Dictonaries for substituted variables: env[] for OS environment vars and var[] for global variables
from IPython.core.magic import register_line_cell_magic
from jinja2 import Template

@register_line_cell_magic
def writetemplate(line, cell):
    dirname = os.path.dirname(line)
    if len(dirname)>0 and not os.path.exists(dirname):
        os.makedirs(dirname)
    with open(line, 'w') as f:
        f.write(Template(cell).render({'env' : os.environ, 'var' : globals()}))

#### 0.1 Global parameters of the training

In [ ]:
# Name of the experiment in MLFlow tracking and name in MLFlow model registry
experiment_name = 'chicago-taxi-m3'
# How many parallel training executing with random training parameters
number_of_parallel_trainings = 3
# Training module version will be composed to 'trainer-0.2'
training_module_version = '0.2'

# Range of randomized RandomForestRegressor estimators lower and upper limits.
range_of_estimators_lower = 20
range_of_estimators_upper = 200

#### 0.2 Print environment variables

In [ ]:
# MLflow public URI
MLFLOW_TRACKING_EXTERNAL_URI = os.environ['MLFLOW_TRACKING_EXTERNAL_URI']

REGION=os.environ['MLOPS_REGION']
ML_IMAGE_URI = os.environ['ML_IMAGE_URI']
COMPOSER_NAME = os.environ['MLOPS_COMPOSER_NAME']
MLFLOW_GCS_ROOT_URI = os.environ['MLFLOW_GCS_ROOT_URI']

print(f'Cloud Composer instance name: {COMPOSER_NAME}')
print(f'Cloud Composer region: {REGION}')
print(f'MLflow tracking server URI: {mlflow.get_tracking_uri()}')
print(f'MLflow GCS root: {MLFLOW_GCS_ROOT_URI}')

experiment_path = MLFLOW_GCS_ROOT_URI.replace('gs://','')

## Step 1: Create training package
### 1.1 Create training package folder and module static content

In [ ]:
!mkdir -p ./package/training
!touch ./package/training/__init__.py

### 1.2 Write setup.py to define package dependencies

In [ ]:
%%writetemplate ./package/setup.py
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['mlflow==1.13.1','PyMySQL==0.9.3']

setup(
    name='taxi-fare-trainer',
    version='{{ var["training_module_version"] }}',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='Custom training setup for Chicago taxi fare prediction.'
)

### 1.3 Create training task Python file

This complex file is the core trainer routine that will be executed in Cloud AI Platform Training environment.
The experimental version of training routine is in 'ChicagoTaxiTrainer.ipynb' where you might adjust and test changes more easily.

#### About the approach
This example is using Scikit-learn [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) in a short training pipeline. 
> Note Cloud AI Platform Training passes input parameters through CLI arguments of main() method. For example the 'number_of_estimators' are passed in arguments list, while this parameter is defined in Airflow DAG BashOperator argument which operator invokes 'gcloud ai-platform jobs submit training' command with 'number_of_estimators'.

Training input parameters are:
* 'number_of_estimators' - The number of trees in the random forest.
* 'max_features' - The number of features to consider when looking for the best split.

Training metrics:
* 'train_cross_valid_score_rmse_mean' - RMSE score on training data split
* 'eval_cross_valid_score_rmse_mean' - RMSE score on test/eval data split

In [ ]:
%%writetemplate ./package/training/task.py

import sys, stat
import argparse
import os
import numpy as np
import pandas as pd
import glob
from scipy import stats

from sklearn.linear_model import LogisticRegression # Only for train_test
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

from joblib import dump, load
from google.cloud import storage

csv_delimiter = '|'

def copy_local_directory_to_gcs(local_path, gcs_uri):
    """ 
    Uploads a local folder structure to a GCS bucket folder. Utilitized to upload trained model to GCS.
    """
    assert os.path.isdir(local_path)
    job_dir =  gcs_uri.replace('gs://', '')
    bucket_id = job_dir.split('/')[0]
    bucket_path = job_dir.lstrip(f'{bucket_id}/')
    bucket = storage.Client().bucket(bucket_id)
    blob = bucket.blob(f'{bucket_path}/{local_path}')
    _upload_local_to_gcs(local_path, bucket, bucket_path)
        
def _upload_local_to_gcs(local_path, bucket, bucket_path):
    """ Recursive file and folder upload from starting folder """
    for local_file in glob.glob(local_path + '/**'):
        if not os.path.isfile(local_file):
           _upload_local_to_gcs(local_file, bucket, bucket_path + '/' + os.path.basename(local_file))
        else:
           remote_path = os.path.join(bucket_path, local_file[1 + len(local_path):])
           blob = bucket.blob(remote_path)
           blob.upload_from_filename(local_file)

def feature_engineering(data):
    """ Prepares the preloaded dataset for final training """
    # Add 'N/A' for missing 'Company'
    data.fillna(value={'company':'N/A','tolls':0}, inplace=True)
    # Drop rows contains null data.
    data.dropna(how='any', axis='rows', inplace=True)
    # Pickup and dropoff locations distance
    data['abs_distance'] = (np.hypot(data['dropoff_latitude']-data['pickup_latitude'], data['dropoff_longitude']-data['pickup_longitude']))*100

    # Drop extremes and outliers
    possible_outliers_cols = ['trip_seconds', 'trip_miles', 'fare', 'abs_distance']
    data = data[(np.abs(stats.zscore(data[possible_outliers_cols])) < 3).all(axis=1)].copy()
    # Reduce location accuracy to improve training speed
    data = data.round({'pickup_latitude': 3, 'pickup_longitude': 3, 'dropoff_latitude':3, 'dropoff_longitude':3})

    # Returns training only features (X) and fare (y)  
    return (
        data.drop(['fare', 'trip_start_timestamp'], axis=1),
        data['fare']
    )

def build_pipeline(number_of_estimators = 20, max_features = 'auto'):
    """ Defines the scikit training steps """
    ct_pipe = ColumnTransformer(transformers=[
    ('hourly_cat', OneHotEncoder(categories=[range(0,24)], sparse = False), ['trip_start_hour']),
    ('dow', OneHotEncoder(categories=[['Mon', 'Tue', 'Sun', 'Wed', 'Sat', 'Fri', 'Thu']], sparse = False), ['trip_start_day_of_week']),
    ('std_scaler', StandardScaler(), [
        'trip_start_year',
        'abs_distance',
        'pickup_longitude',
        'pickup_latitude',
        'dropoff_longitude',
        'dropoff_latitude',
        'trip_miles',
        'trip_seconds'])
    ])
    rfr_pipe = Pipeline([
        ('ct', ct_pipe),
        ('forest_reg', RandomForestRegressor(n_estimators = number_of_estimators, max_features = max_features, n_jobs = -1, random_state = 3))
    ])
    return rfr_pipe

def train_model(args):
    """ Main training logic """
    print('Taxi fare estimation model training step started...')
    # Addresses experiment by name for following tracking context
    mlflow.set_experiment(args.experiment_name)
    
    # To save training parameters and metrics automatically use autolog()
    # mlflow.sklearn.autolog()
    with mlflow.start_run(nested=True) as mlflow_run:
        mlflow.log_param('number_of_estimators', args.number_of_estimators)
        mlflow.set_tag('version', args.version_tag)
        mlflow.set_tag('job_name', args.job_name)
        mlflow.set_tag('gcs_train_source', args.gcs_train_source)
        mlflow.set_tag('gcs_eval_source', args.gcs_eval_source)

        df = pd.read_csv(args.gcs_train_source, sep=csv_delimiter)
        mlflow.log_param('training_size', f'{df.shape[0]}')
        
        # Fixing and droping invalid data rows.
        X_train, y_train = feature_engineering(df)
        # Create training pipeline.
        rfr_pipe = build_pipeline(number_of_estimators=args.number_of_estimators)
        
        rfr_score = cross_val_score(rfr_pipe, X_train, y_train, scoring = 'neg_mean_squared_error', cv=5)
        mlflow.log_metric('train_cross_valid_score_rmse_mean', np.sqrt(-rfr_score).mean())
        
        # Train the model
        final_model = rfr_pipe.fit(X_train, y_train)
        mlflow.sklearn.log_model(final_model, 'chicago_rnd_forest')

        # Evaluate model to eval set
        df = pd.read_csv(args.gcs_eval_source, sep=csv_delimiter)
        mlflow.log_param('eval_size',f'{df.shape[0]}')
        X_eval, y_eval = feature_engineering(df)
        X_eval['fare_pred'] = final_model.predict(X_eval)
        rfr_score = cross_val_score(final_model, X_eval, y_eval, scoring='neg_mean_squared_error', cv=5)
        mlflow.log_metric('eval_cross_valid_score_rmse_mean', np.sqrt(-rfr_score).mean())
        
        # Save model
        model_file_name = f'{args.version_tag}.joblib'
        mlflow.sklearn.save_model(final_model, model_file_name)
        copy_local_directory_to_gcs(model_file_name, args.job_dir)
        mlflow.set_tag('model_file', args.job_dir+'/'+model_file_name)

    print('Training finished.')

def main():
    print('Training arguments: ' + ' '.join(sys.argv[1:]))
    parser = argparse.ArgumentParser()
    parser.add_argument('--number_of_estimators', type=int)
    parser.add_argument('--job-dir', type=str)
    parser.add_argument('--local_data', type=str)
    parser.add_argument('--gcs_train_source', type=str)
    parser.add_argument('--gcs_eval_source', type=str)
    parser.add_argument('--experiment_name', type=str)
    parser.add_argument('--version_tag', type=str)
    parser.add_argument('--job_name', type=str)
    
    args, unknown_args = parser.parse_known_args()

    if not args.gcs_train_source:
        print('Missing GCS training source URI')
        return
    if not args.gcs_eval_source:
        print('Missing GCS evaluation source URI')
        return
    # CLOUD_ML_JOB conatains other CAIP Training runtime parameters in JSON object
    # job = os.environ['CLOUD_ML_JOB']
    
    # MLflow locally available
    mlflow.set_tracking_uri('http://127.0.0.1:80')

    train_model(args)

if __name__ == '__main__':
    main()

### 1.4 Package the training modules
Create training package and import(copy) to the Cloud Composer's 'data' folder, since than the compressed package file will be available from DAG code.

In [ ]:
# Create trainer packege
!cd package && python ./setup.py sdist

# Copy to Composer data folder
!gcloud composer environments storage data import \
    --environment {COMPOSER_NAME} \
    --location {REGION} \
    --source ./package/dist \
    --destination multi_model_trainer_dag

## Step 2: Create model trainer Airflow DAG
Notice: The entire cell is a template will be written to 'multi_model_trainer_dag.py' file.
        'writetemplate' magic uses Jinja templating while Airflow also provides Jinja templating for runtime parameters.
Airflow parameters should be wrapped in this syntax: {{ "{{ ts_nodash }}" }} because of 'the template in the template' mechanizm.

### 2.1 Write out Cloud Composer/Airflow DAG file

In [ ]:
%%writetemplate multi_model_trainer_dag.py

import os
import logging
import random
import uuid
from datetime import (datetime, timedelta)

import mlflow
import mlflow.sklearn

import airflow
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
from airflow.contrib.operators.bigquery_operator import BigQueryOperator
from airflow.contrib.operators.bigquery_table_delete_operator import BigQueryTableDeleteOperator
from airflow.contrib.operators.bigquery_to_gcs import BigQueryToCloudStorageOperator
from airflow.providers.google.cloud.operators.mlengine import MLEngineStartTrainingJobOperator

csv_delimiter = '|'
experiment_name = '{{ var["experiment_name"] }}'
ML_IMAGE_URI = '{{ var["ML_IMAGE_URI"] }}'
job_experiment_root = f'{{ var["MLFLOW_GCS_ROOT_URI"] }}/experiments/{experiment_name}'

PROJECT_ID = os.getenv('GCP_PROJECT', default='edgeml-demo')
REGION = os.getenv('COMPOSER_LOCATION', default='us-central1')

# Postfixes for temporary BQ tables and output CSV files
TRAINING_POSTFIX = '_training'
EVAL_POSTFIX = '_eval'
VALIDATION_POSTFIX = '_validation'

BQ_DATASET = 'chicago_taxi_trips'
BQ_TABLE = 'taxi_trips'

# Query to create training and evaluation dataset from public taxi_trips table.
# Some key aspects:
# - Localize trip dates to Chicago time zone.
# - Create a hint (is_airport) to categorize airport to/from travel cases.
# - Filter out inapropiate rows (null or zero values)
# - Add training features for less granuated year, month, day, hour and day_of_week instead of using compound date time field 
BQ_QUERY = """
with tmp_table as (
SELECT trip_seconds, trip_miles, fare, tolls, 
    company, pickup_latitude, pickup_longitude, dropoff_latitude, dropoff_longitude,
    DATETIME(trip_start_timestamp, 'America/Chicago') trip_start_timestamp,
    DATETIME(trip_end_timestamp, 'America/Chicago') trip_end_timestamp,
    CASE WHEN (pickup_community_area IN (56, 64, 76)) OR (dropoff_community_area IN (56, 64, 76)) THEN 1 else 0 END is_airport,
FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
WHERE
    dropoff_latitude IS NOT NULL and
    dropoff_longitude IS NOT NULL and
    pickup_latitude IS NOT NULL and
    pickup_longitude IS NOT NULL and
    fare > 0 and 
    trip_miles > 0 and
    MOD(ABS(FARM_FINGERPRINT(unique_key)), 100) {}
ORDER BY RAND()
LIMIT {})
SELECT *,
    EXTRACT(YEAR FROM trip_start_timestamp) trip_start_year,
    EXTRACT(MONTH FROM trip_start_timestamp) trip_start_month,
    EXTRACT(DAY FROM trip_start_timestamp) trip_start_day,
    EXTRACT(HOUR FROM trip_start_timestamp) trip_start_hour,
    FORMAT_DATE('%a', DATE(trip_start_timestamp)) trip_start_day_of_week
FROM tmp_table
"""

def joiner_func(training_gcs_file_name, eval_gcs_file_name, **kwargs):
    """
    No-ops method to synchonize pipeline braches
    """
    logging.info('Joining %s, eval GCS files %s', training_gcs_file_name, eval_gcs_file_name)
    return None

def fake_model_tracking(**kwargs):
    """
    Simulated training, if you want to test environment without real time consuming training,
    but want to see random metrics in MLflow
    """
    job_name = kwargs.get('templates_dict').get('job_name')
    print(f"Fake model tracking: '{job_name}'")
    mlflow.set_experiment(experiment_name)
    with mlflow.start_run(nested=True) as run:
        mlflow.log_param('number_of_estimators', 0)
        mlflow.set_tag('version', 'fake')
        mlflow.set_tag('job_name', job_name)
        mlflow.log_metric('train_cross_valid_score_rmse_mean', 1+random.random())
        mlflow.log_metric('eval_cross_valid_score_rmse_mean', 1+random.random())
    return None

def register_model(run_id, model_name):
    """
    Register model to MLflow
    """
    model_uri = f'runs:/{run_id}/{model_name}'
    registered_model = mlflow.register_model(model_uri, model_name)
    print(registered_model)

def compare_to_registered_model(model_name, best_run, metric_to_compare):
    """
    Compare the actual training results with latest registered model.
    Latest registered model is the previous best.
    """
    mlflow_client = mlflow.tracking.MlflowClient()
    registered_models=mlflow_client.search_registered_models(filter_string=f"name='{model_name}'", max_results=1, order_by=['timestamp DESC'])
    if len(registered_models)==0:
        # No previous training job.
        register_model(best_run.run_id, model_name)
    else:
        last_version = registered_models[0].latest_versions[0]
        run = mlflow_client.get_run(last_version.run_id)
        if not run:
            print(f'Registered version run missing!')            
            return None
        
        # Suppose the las registered model is the best in the training history
        last_registered_metric=run.data.metrics[metric_to_compare]
        best_run_metric=best_run['metrics.'+metric_to_compare]
        # Smaller value is better
        if last_registered_metric>best_run_metric:
            print(f'Register better version with metric: {best_run_metric}')
            register_model(best_run.run_id, experiment_name)
        else:
            print(f'Registered version still better. Metric: {last_registered_metric}')    

def model_blessing(**kwargs):
    """
    Compare all paralell training and select the best on 'eval_cross_valid_score_rmse_mean' base.
    """
    job_name = kwargs.get('templates_dict').get('job_name')
    print(f'Model blessing: "{job_name}"')

    # Query results of current training jobs 
    experiment = mlflow.get_experiment_by_name(experiment_name)
    filter_string = f"tags.job_name ILIKE '{job_name}_%'"
    df = mlflow.search_runs([experiment.experiment_id], filter_string=filter_string)

    # Compare new trained model and select the best.
    eval_max = df.loc[df['metrics.eval_cross_valid_score_rmse_mean'].idxmax()]
    # train_max can be an alternative for comparizon.
    # train_max= df.loc[df['metrics.train_cross_valid_score_rmse_mean'].idxmax()]
    
    compare_to_registered_model(experiment_name, eval_max, 'eval_cross_valid_score_rmse_mean')

with DAG('multi_model_trainer',
         description = 'Train evaluate and validate multi models on taxi fare dataset. Select the best one and register it to MLflow v0.1',
         schedule_interval = '*/15 * * * *', # '*/15 ...' -> every 15 minutes,  None -> manual trigger
         start_date = datetime(2021, 1, 1),
         max_active_runs = 3,
         catchup = False,
         default_args = { 'provide_context': True}
         ) as dag:

    # Dataset split ratio and limit of query records
    tasks = {
        'training' : {
            'dataset_range' : 'between 0 and 80',
            'limit' : random.randrange(2000, 8000, 100)
            },
        'eval':{
            'dataset_range' : 'between 80 and 100',
            'limit' : random.randrange(1000, 2000, 100)
        }}

    # Define task list for preparation
    for task_key in tasks.keys():
        # Note: fix table names causes race condition in case when DAG triggered before the previous finished.
        table_name = f'{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE}_{task_key}'
        task = tasks[task_key]
        task['gcs_file_name'] = f'{job_experiment_root}/data/ds_{task_key}.csv'
        
        # Deletes previous training temporary tables
        task['delete_table'] = BigQueryTableDeleteOperator(
            task_id = 'delete_table_' + task_key,
            deletion_dataset_table = table_name,
            ignore_if_missing = True)

        # Splits and copy source BQ table to 'dataset_range' sized segments
        task['split_table'] = BigQueryOperator(
            task_id = 'split_table_' + task_key,
            use_legacy_sql=False,
            destination_dataset_table = table_name,
            sql = BQ_QUERY.format(task['dataset_range'],task['limit']),
            location = REGION)
        
        # Extract split tables to CSV files in GCS
        task['extract_to_gcs'] = BigQueryToCloudStorageOperator(
            task_id = 'extract_to_gcs_' + task_key,
            source_project_dataset_table = table_name,
            destination_cloud_storage_uris = [task['gcs_file_name']],
            field_delimiter = csv_delimiter)

    joiner_1 = PythonOperator(
        task_id = 'joiner_1',
        python_callable = joiner_func,
        op_kwargs={ 'training_gcs_file_name': tasks['training']['gcs_file_name'],
                    'eval_gcs_file_name': tasks['eval']['gcs_file_name']})

    # Create an unique job name
    submit_time = datetime.now().strftime('%Y%m%d_%H%M%S')
    job_name = f'training_job_{submit_time}'
    job_dir = f'{job_experiment_root}/dmt_{submit_time}'
    
    # Train model in Cloud AI Platform Training.
    # To run training job we have 3 options:
    # 1 - gcloud CLI command with Bashoperator
    # 2 - API client (https://cloud.google.com/ai-platform/training/docs/python-client-library) from Python operator
    # 3 - Native MLEngineStartTrainingJobOperator Airflow operator
    #
    # In this example we are using the 1st option, because this is the only way to
    # declare custom trainer Docker image.
    
    # Template for string format ({variable}) and jinja template ({{variable}})
    training_command_tmpl="""gcloud ai-platform jobs submit training {job_name} \
        --region {region} \
        --scale-tier BASIC \
        --job-dir {job_dir} \
        --package-path /home/airflow/gcs/data/multi_model_trainer_dag/package/training/ \
        --module-name training.task \
        --master-image-uri {ml_image_uri} \
        --stream-logs \
        -- \
        --experiment_name {experiment_name} \
        --gcs_train_source {gcs_train_source} \
        --gcs_eval_source {gcs_eval_source} \
        --version_tag {version_tag} \
        --number_of_estimators {number_of_estimators} \
        --job_name {job_name}"""

    training_tasks = []
    for training_id in range(0, {{var['number_of_parallel_trainings']}}):
        # Simulated training, if you want to test environment without real training but random metrics
        # trainer = PythonOperator(
        #    task_id = f'trainer_{training_id}',
        #    python_callable = fake_model_tracking,
        #    templates_dict={'job_name': 'training_job_{{ "{{ ts_nodash }}" }}'+f'_{training_id}'})
        
        trainer = BashOperator(
            task_id=f'trainer_{training_id}',
            bash_command=training_command_tmpl.format(
                 region = REGION,
                 job_name = 'training_job_{{ "{{ ts_nodash }}" }}'+f'_{training_id}',
                 job_dir = job_dir+f'_{training_id}',
                 ml_image_uri = ML_IMAGE_URI,
                 gcs_train_source = tasks['training']['gcs_file_name'],
                 gcs_eval_source = tasks['eval']['gcs_file_name'],
                 experiment_name = experiment_name,
                 version_tag = f'trainer_{training_id}',
                 # The only difference in trainings:
                 number_of_estimators = random.randrange({{var['range_of_estimators_lower']}} , {{var['range_of_estimators_upper']}}))
        )

        training_tasks.append(trainer)
    
    # Select the best model of this run
    model_blessing = PythonOperator(
        task_id = 'model_blessing',
        python_callable = model_blessing,
        templates_dict={'job_name': 'training_job_{{ "{{ ts_nodash }}" }}'})

    # Exectute tasks
    for task_key, task in tasks.items():
        task['delete_table'] >> task['split_table'] >> task['extract_to_gcs'] >> joiner_1

    # Brancing and merging training tasks
    for trainer in training_tasks:
        trainer.set_upstream(joiner_1)
        model_blessing.set_upstream(trainer)

### 2.2 Copy DAG file to Cloud Composer

In [ ]:
!gcloud composer environments storage dags import \
  --environment {COMPOSER_NAME}  \
  --location {REGION} \
  --source multi_model_trainer_dag.py

### 2.3 Start training pipeline

* Navigate to Cloud Composer, click on 'Airflow' in [Composer environment list](http://console.cloud.google.com/composer/environments)
* Start pipeline by enable it (off->on) 
* You will see the pipeline progresses in 'Tree View' page
* Check training jobs in https://pantheon.corp.google.com/ai-platform/jobs
* When all jobs and pipeline finishes, you can check results of this test in MLflow and GCS folder

(next cell creates links to MLflow and GCS)

In [ ]:
display(HTML(f'<h4><a href="{MLFLOW_TRACKING_EXTERNAL_URI}" rel="noopener noreferrer" target="_blank">Open MLflow UI and check metrics</a></h4>'))
display(HTML(f'<h4><a href="https://console.cloud.google.com/storage/browser/{experiment_path}/experiments/{experiment_name}" rel="noopener noreferrer" target="_blank">Open "{experiment_name}" experiment folder in GCS</a></h4>'))
